In [1]:
import $file.^.sparkinit, sparkinit._
import $file.^.pathinit, pathinit._
import ss.implicits._
import org.apache.spark.sql.functions._
import java.nio.file.Paths
import org.apache.spark.storage.StorageLevel

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
19/12/16 16:03:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import $file.$          , sparkinit._

import $file.$         , pathinit._

import ss.implicits._

import org.apache.spark.sql.functions._

import java.nio.file.Paths

import org.apache.spark.storage.StorageLevel

In [2]:
val input_dir = Paths.get(EXTRACT_DIR.resolve("gene.json").toString)
val df = ss.read.json(input_dir.toString)
df

json at cmd1.sc:2

16 / 16

input_dir: java.nio.file.Path = /home/eczech/data/ot/extract/gene.json
df: org.apache.spark.sql.package.DataFrame = [_private: struct<facets: struct<reactome: struct<pathway_code: array<string>, pathway_type_code: array<string>>>, suggestions: struct<input: array<string>, output: string ... 1 more field>>, alias_name: array<string> ... 60 more fields]
res1_2: org.apache.spark.sql.package.DataFrame = [_private: struct<facets: struct<reactome: struct<pathway_code: array<string>, pathway_type_code: array<string>>>, suggestions: struct<input: array<string>, output: string ... 1 more field>>, alias_name: array<string> ... 60 more fields]

In [18]:
val dft = df.filter(size($"dbxrefs") > 0).select("id", "dbxrefs").persist()

dft: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, dbxrefs: array<string>]

In [21]:
dft.selectExpr("transform(dbxrefs, v -> v)").show(3)

show at cmd20.sc:1

1 / 1

+--------------------------------------------------------------------------------+
|transform(dbxrefs, lambdafunction(namedlambdavariable(), namedlambdavariable()))|
+--------------------------------------------------------------------------------+
|                                                            [Bgee:ENSG0000021...|
|                                                            [Bgee:ENSG0000000...|
|                                                            [Bgee:ENSG0000022...|
+--------------------------------------------------------------------------------+
only showing top 3 rows



In [22]:
import org.apache.spark.sql.Column

import org.apache.spark.sql.Column

In [31]:
def parse(col: String): Column = {
    val q =       """
        |transform(%s, v ->
        | case when v like "%%/identifiers.org/eco/%%" 
        |   then regexp_replace(element_at(split(v, "/"), -1), "ECO:", "ECO_")
        | else element_at(split(v, ":"), 1)
        | end)
        |""".stripMargin.format(col)
    println(q)
    expr(q)
}
dft.select(parse("dbxrefs")).show(3)


transform(dbxrefs, v ->
 case when v like "%/identifiers.org/eco/%" 
   then regexp_replace(element_at(split(v, "/"), -1), "ECO:", "ECO_")
 else element_at(split(v, ":"), 1)
 end)



show at cmd30.sc:12

1 / 1

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|transform(dbxrefs, lambdafunction(CASE WHEN namedlambdavariable() LIKE %/identifiers.org/eco/% THEN regexp_replace(element_at(split(namedlambdavariable(), /), -1), ECO:, ECO_) ELSE element_at(split(namedlambdavariable(), :), 1) END, namedlambdavariable()))|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                                                              

defined function parse

In [2]:
val input_dir = Paths.get(EXTRACT_DIR.resolve("evidence_raw.json").toString)
val df = ss.read.json(input_dir.toString)
df

json at cmd1.sc:2

178 / 178

19/12/16 14:23:45 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


input_dir: java.nio.file.Path = /home/eczech/data/ot/extract/evidence_raw.json
df: org.apache.spark.sql.package.DataFrame = [_corrupt_record: string, access_level: string ... 10 more fields]
res1_2: org.apache.spark.sql.package.DataFrame = [_corrupt_record: string, access_level: string ... 10 more fields]

## Source EDA

In [3]:
df.groupBy("sourceID", "type").count().show

show at cmd2.sc:1

9 / 9

show at cmd2.sc:1

1 / 1

+------------------+-------------------+------+
|          sourceID|               type| count|
+------------------+-------------------+------+
|         phenodigm|       animal_model|502550|
|            chembl|         known_drug|400440|
|  expression_atlas|     rna_expression|213490|
|      gwas_catalog|genetic_association|194170|
|               eva|genetic_association|103358|
|       eva_somatic|   somatic_mutation|  7889|
|cancer_gene_census|   somatic_mutation| 96284|
|    phewas_catalog|genetic_association| 71027|
|        slapenrich|   affected_pathway| 74583|
|uniprot_literature| genetic_literature|  4843|
|           uniprot|genetic_association| 31665|
|   uniprot_somatic|   somatic_mutation|   284|
|  genomics_england| genetic_literature| 11079|
|          reactome|   affected_pathway| 11049|
|    gene2phenotype| genetic_literature|  1603|
|         europepmc|         literature|  1000|
|           intogen|   somatic_mutation|  2594|
|            crispr|   affected_pathway|

## ECO EDA

In [3]:
df.groupBy($"evidence.evidence_codes".isNull).count.show

show at cmd2.sc:1

178 / 178

show at cmd2.sc:1

1 / 1

+---------------------------------+-------+
|(evidence.evidence_codes IS NULL)|  count|
+---------------------------------+-------+
|                             true|1303211|
|                            false|9240849|
+---------------------------------+-------+



In [19]:
val dft = Seq(List(1),List(2),List(3,4), List(3,4,4,4)).toDF
    .withColumn("s", when(size($"value") > 1, lit(Array(1))))
dft.show
dft
    .withColumn("t", concat($"value", coalesce($"s", array())))
    .withColumn("x", array_distinct(concat($"value", coalesce($"s", array()))))
    .show

+------------+----+
|       value|   s|
+------------+----+
|         [1]|null|
|         [2]|null|
|      [3, 4]| [1]|
|[3, 4, 4, 4]| [1]|
+------------+----+



dft: org.apache.spark.sql.package.DataFrame = [value: array<int>, s: array<int>]

In [22]:
val dfeco = df
    .withColumn(
        "evidence_codes_source", 
        when($"evidence.evidence_codes".isNotNull, lit("base"))
        .when($"evidence.variant2disease".isNotNull, lit("v2d"))
        .when($"evidence.target2drug".isNotNull, lit("t2d"))
        .when($"evidence.biological_model".isNotNull, lit("biom"))
    )
    .withColumn(
        "evidence_codes_aggregate", 
        array_distinct(
            when($"evidence_codes_source" === "base", $"evidence.evidence_codes")
            .when($"evidence_codes_source" === "v2d", concat(
                coalesce($"evidence.variant2disease.evidence_codes", array()),
                coalesce($"evidence.gene2variant.evidence_codes", array())
            ))
            .when($"evidence_codes_source" === "t2d", concat(
                coalesce($"evidence.target2drug.evidence_codes", array()),
                coalesce($"evidence.drug2clinic.evidence_codes", array())
            ))
            .when($"evidence_codes_source" === "biom", 
                coalesce($"evidence.biological_model.evidence_codes", array())
            )
            .otherwise(array())
        )
    )

dfeco: org.apache.spark.sql.package.DataFrame = [_corrupt_record: string, access_level: string ... 12 more fields]

In [23]:
dfeco.groupBy("evidence_codes_source").count.show

show at cmd22.sc:1

178 / 178

show at cmd22.sc:1

1 / 1

+---------------------+-------+
|evidence_codes_source|  count|
+---------------------+-------+
|                 biom| 502550|
|                  v2d| 400220|
|                 base|9240849|
|                  t2d| 400440|
|                 null|      1|
+---------------------+-------+



In [39]:
val dfex = dfeco
    .select($"evidence_codes_source", explode($"evidence_codes_aggregate").as("col"))
    .dropDuplicates()
    .withColumn("parts", struct(
        element_at(split($"col", "/"), 3).as("domain"),
        element_at(split($"col", "/"), 4).as("dir"),
        element_at(split($"col", "/"), 5).as("name")
    ))
    .persist(StorageLevel.DISK_ONLY)
dfex.show(3, false)

show at cmd38.sc:10

178 / 178

show at cmd38.sc:10

1 / 1

+---------------------+------------------------------------------------+---------------------------------------------+
|evidence_codes_source|col                                             |parts                                        |
+---------------------+------------------------------------------------+---------------------------------------------+
|biom                 |http://identifiers.org/eco/ECO:0000179          |[identifiers.org, eco, ECO:0000179]          |
|v2d                  |http://purl.obolibrary.org/obo/ECO_0000205      |[purl.obolibrary.org, obo, ECO_0000205]      |
|v2d                  |http://identifiers.org/eco/cttv_mapping_pipeline|[identifiers.org, eco, cttv_mapping_pipeline]|
+---------------------+------------------------------------------------+---------------------------------------------+
only showing top 3 rows



dfex: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [evidence_codes_source: string, col: string ... 1 more field]

In [44]:
dfex.count

count at cmd43.sc:1

1 / 1

res43: Long = 19L

In [40]:
dfex.groupBy("parts.domain").count.show(10, false)

show at cmd39.sc:1

1 / 1

show at cmd39.sc:1

1 / 1

+------------------------+-----+
|domain                  |count|
+------------------------+-----+
|identifiers.org         |6    |
|purl.obolibrary.org     |12   |
|www.targetvalidation.org|1    |
+------------------------+-----+



In [45]:
dfex.groupBy("parts.domain", "parts.dir","parts.name").count.show(100, false)

show at cmd44.sc:1

1 / 1

show at cmd44.sc:1

1 / 1

+------------------------+--------+---------------------+-----+
|domain                  |dir     |name                 |count|
+------------------------+--------+---------------------+-----+
|identifiers.org         |eco     |ECO:0000179          |1    |
|purl.obolibrary.org     |obo     |ECO_0000205          |3    |
|identifiers.org         |eco     |cttv_mapping_pipeline|1    |
|www.targetvalidation.org|evidence|literature_mining    |1    |
|purl.obolibrary.org     |obo     |ECO_0000213          |1    |
|identifiers.org         |eco     |target_drug          |1    |
|identifiers.org         |eco     |drug_disease         |1    |
|identifiers.org         |eco     |GWAS                 |1    |
|purl.obolibrary.org     |obo     |ECO_0000058          |1    |
|purl.obolibrary.org     |obo     |ECO_0000295          |1    |
|purl.obolibrary.org     |obo     |ECO_0000053          |1    |
|identifiers.org         |eco     |PheWAS               |1    |
|purl.obolibrary.org     |obo     |ECO_0

In [42]:
dfex.groupBy(
    $"parts.domain", 
    $"parts.name".contains("_"), 
    $"parts.name".contains(":")
).count.show

show at cmd41.sc:4

1 / 1

show at cmd41.sc:4

1 / 1

+--------------------+-----------------------+-----------------------+-----+
|              domain|contains(parts.name, _)|contains(parts.name, :)|count|
+--------------------+-----------------------+-----------------------+-----+
|     identifiers.org|                  false|                   true|    1|
| purl.obolibrary.org|                   true|                  false|   12|
|     identifiers.org|                   true|                  false|    3|
|www.targetvalidat...|                   true|                  false|    1|
|     identifiers.org|                  false|                  false|    2|
+--------------------+-----------------------+-----------------------+-----+



In [43]:
dfex.filter($"parts.name".contains("_") && $"parts.domain" =!= "identifiers.org")
    .show(10, false)

show at cmd42.sc:2

1 / 1

+---------------------+----------------------------------------------------------+-------------------------------------------------------+
|evidence_codes_source|col                                                       |parts                                                  |
+---------------------+----------------------------------------------------------+-------------------------------------------------------+
|v2d                  |http://purl.obolibrary.org/obo/ECO_0000205                |[purl.obolibrary.org, obo, ECO_0000205]                |
|base                 |http://purl.obolibrary.org/obo/ECO_0000205                |[purl.obolibrary.org, obo, ECO_0000205]                |
|base                 |http://www.targetvalidation.org/evidence/literature_mining|[www.targetvalidation.org, evidence, literature_mining]|
|base                 |http://purl.obolibrary.org/obo/ECO_0000213                |[purl.obolibrary.org, obo, ECO_0000213]                |
|t2d                  |http